In [188]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVR

In [163]:
df = pd.read_csv('../Data_Cleaning/Data/gurgaon_properties_post_feature_selection.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [164]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,99.0,4.0,5.0,4.0,3.0,2547.0,0.0,0.0,1.0,1.0,2.0,2.20
1,0.0,79.0,2.0,2.0,3.0,3.0,1130.0,0.0,0.0,0.0,1.0,1.0,1.08
2,0.0,113.0,2.0,2.0,2.0,3.0,1046.0,0.0,0.0,1.0,0.0,2.0,0.99
3,1.0,47.0,10.0,10.0,4.0,3.0,3123.0,0.0,0.0,1.0,0.0,1.0,8.00
4,0.0,17.0,4.0,5.0,4.0,3.0,2650.0,1.0,0.0,0.0,2.0,0.0,3.70


In [165]:
# One hot encode -> sector, balacony, agePossession, furnishing type, luxury category, floor category

In [166]:
X = df.drop(columns=['price'])
y = df['price']

In [167]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [168]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [169]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [170]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [189]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [190]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

C:\Users\amrit\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\amrit\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [191]:
scores.mean()

0.8892986247310708

In [192]:
scores.std()

0.013047128163839168

In [193]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

In [194]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [195]:
y_pred = pipeline.predict(X_test)

In [196]:
y_pred = np.expm1(y_pred)

In [197]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5155829459135702